## Qiskit Nature é uma biblioteca com o objetivo de calcular energias moleculares em computadores quânticos

In [3]:
# Molecule é uma classe que do qiskit Nature que implementa a geometria de uma molécula
# UnitsType is used to inform if the units are in Angstrom or Bohr
from qiskit_nature.drivers import Molecule, UnitsType

# Drivers to resolve the electronic structure of a molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem

# Convert the molecular electronic structure into qubit circuits
from qiskit_nature.converters.second_quantization import QubitConverter

# The Jordan-Wigner fermion-to-qubit mapping
from qiskit_nature.mappers.second_quantization import JordanWignerMapper


In [5]:
# Pass the molecule geometry (Ex. H2)

molecule = Molecule(geometry=[['H', [0., 0., 0.]],
                              ['H', [0., 0., 0.735]]],
                     charge=0, multiplicity=1)

driver = ElectronicStructureMoleculeDriver(
    molecule,
    basis='sto3g',
    driver_type=ElectronicStructureDriverType.PYSCF
)

es_problem = ElectronicStructureProblem(driver)
qubit_converter = QubitConverter(JordanWignerMapper())

In [6]:
# Importing the solver for the circuit built with the JordanWignerMapper
from qiskit_nature.algorithms import NumPyEigensolverFactory

numpy_solver = NumPyEigensolverFactory(use_default_filter_criterion=True)

In [7]:
# To find the excited states, we need to calculate the ground state
# with the GroundStateEigensolver

from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_nature.algorithms import GroundStateEigensolver, QEOM, VQEUCCFactory

# This first part sets the ground state solver
# see more about this part in the ground state calculation tutorial
quantum_instance = QuantumInstance(Aer.get_backend('aer_simulator_statevector'))
solver = VQEUCCFactory(quantum_instance)
gsc = GroundStateEigensolver(qubit_converter, solver)

# The qEOM algorithm is simply instantiated with the chosen ground state solver
qeom_excited_states_calculation = QEOM(gsc, 'sd')

In [8]:
from qiskit_nature.algorithms import ExcitedStatesEigensolver

numpy_excited_states_calculation = ExcitedStatesEigensolver(qubit_converter, numpy_solver)
numpy_results = numpy_excited_states_calculation.solve(es_problem)

qeom_results = qeom_excited_states_calculation.solve(es_problem)

print(numpy_results)
print('\n\n')
print(qeom_results)

MissingOptionalLibraryError: "The 'PySCF' library is required to use 'PySCFDriver'. You can install it with 'pip install 'qiskit-nature[pyscf]''.  See https://pyscf.org/install.html."